# Project 5: Predicting Power Outage Causes

**Name(s)**: Nicole Liu

**Website Link**: https://nliu880.github.io/Predicting-Power-Outage-Cause/

## Code

In [31]:
# imports <3

import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, LabelEncoder, QuantileTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

import matplotlib.pyplot as plt
import plotly.express as px
pd.options.plotting.backend = 'plotly'

### Framing the Problem

This project will focus on the same dataset used in Project 3, and will aim to use prediction models to solve some form of prediction problem. The problem I have chosen to focus on in this report is predicting the cause of a power outage based on other features.

I will use a multiclass classification model to try and solve this prediction problem. The response variable will be `cause category`; this variable describes the general cause (ex. severe weather or intentional attack) of some power outage, which is what I am trying to predict. My evaluation metric will be the F-1 score. I want to balance both precision and recall. Accuracy in this case could also be used, but from my previous EDA in Project 3, I discovered there were large outliers that skewed the data. We want to avoid this in our metric, so we will use the F-1 score. 

We will begin this by performing the same data cleaning steps as from Project 3. The code used to clean the data below is from there and condensed into one cell. I have made a few edits that involves adding the time unit to the `outage duration` columns. One is in minutes (the original) and ones is in hours (which I added in for clarity and intuition). I have also dropped the `cause category detail`, `hurricane name`, and `postal code` columns. In addition, we also fill in the missing values for certain columns by imputing either the mean or the mode of the group when grouped by `cause category`.

In [25]:
# Reading in the dataframe. The original file came in as an Excel sheet, and thus had to be modified for use, hence
# the "header = 5" portion. 

full_df = pd.read_csv('outage.csv', header = 5) 
columns = full_df.columns.values[4:20] 
columns = np.append(columns, 'POPULATION')

df = pd.read_csv('outage.csv', header = 5, usecols = columns) # re-reading the dataframe
df.drop([0], inplace = True) # dropping the row that contained units for certain columns

# more intense cleaning of data in set

# renaming all the columns, so i don't have to keep typing in all caps

new_cols = {i: i.lower().replace('.', ' ') for i in df.columns.values}

df.rename(columns = {'U.S._STATE': 'us state'}, inplace = True)
df.rename(columns = new_cols, inplace = True)

# we should typecast all numerical information to either an int or float format; object type is not useful

df['anomaly level'] = df['anomaly level'].astype(float)
df['outage duration'] = df['outage duration'].astype(float)
df['demand loss mw'] = df['demand loss mw'].astype(float)
df['customers affected'] = df['customers affected'].astype(float)
df['population'] = df['population'].astype(float)


# coverting the outage start and end time into the appropriate datetime objects

df['outage start date'] = pd.to_datetime(df['outage start date']).dt.date
df['outage start time'] = pd.to_datetime(df['outage start time']).dt.time
df['outage restoration date'] = pd.to_datetime(df['outage restoration date']).dt.date
df['outage restoration time'] = pd.to_datetime(df['outage restoration time']).dt.time

df = df.reset_index()

# helper functions to convert the outage start info into one column and the restoration info into one column

def datetime_start(x):
    date = str(df.loc[x - 1, 'outage start date'])
    time = str(df.loc[x - 1, 'outage start time'])
    
    if (date != 'NaT') & (time != 'NaT'):
        return pd.to_datetime(date + ' ' + time)
    else:
        return np.NaN
    
def datetime_end(x):
    date = str(df.loc[x - 1, 'outage restoration date'])
    time = str(df.loc[x - 1, 'outage restoration time'])
    
    if (date != 'NaT') & (time != 'NaT'):
        return pd.to_datetime(date + ' ' + time)
    else:
        return np.NaN
    
# adding new columns for the combined date and time for outage start and restoration, as well as total outage time

df['start time'] = df['index'].apply(datetime_start)
df['restoration time'] = df['index'].apply(datetime_end)
df['total time'] = df['restoration time'] - df['start time']

# converting the 'outage duration' column that was given from minutes to hours
df['outage duration (hrs)'] = np.round(df['outage duration'] / 60, 2)
df.rename(columns = {'outage duration': 'outage duration (min)'}, inplace = True)

# dropping columns that we no longer need

df.drop(['index', 'outage start date', 'outage start time', 'outage restoration date', 'outage restoration time'], 
        axis = 1, inplace = True)

df.drop(['cause category detail', 'hurricane names', 'postal code'], axis = 1, inplace = True)


In [26]:
print(df.shape)
df.isna().sum()

# we check to see how many missing values there are in each column, as well as the number of observations total

(1534, 14)


us state                   0
nerc region                0
climate region             6
anomaly level              9
climate category           9
cause category             0
outage duration (min)     58
demand loss mw           705
customers affected       443
population                 0
start time                 9
restoration time          58
total time                58
outage duration (hrs)     58
dtype: int64

In [27]:
# we will fill in those missing values for outage duration (min) and anomaly level by taking the mean value as 
# grouped by cause and filling them in

# we will also fill in missing values for climate region and climate category by picking the most common value
# when grouped by category

means = df.groupby('cause category').mean()
mode = df.groupby('cause category').agg(pd.Series.mode)

miss_mean_ind = df[(df['outage duration (min)'].isna()) | (df['anomaly level'].isna())].index
miss_mode_ind = df[(df['climate region'].isna()) | (df['climate category'].isna())].index

for i in miss_mean_ind:
    cause = df.loc[i]['cause category']
    df.loc[i, 'outage duration (min)'] = means.loc[cause]['outage duration (min)']
    df.loc[i, 'outage duration (hrs)'] = means.loc[cause]['outage duration (hrs)']
    df.loc[i, 'anomaly level'] = means.loc[cause]['anomaly level']
    
for i in miss_mode_ind:
    cause = df.loc[i]['cause category']
    df.loc[i, 'climate region'] = mode.loc[cause]['climate region']
    df.loc[i, 'climate category'] = mode.loc[cause]['climate category']

    
df.isna().sum()

us state                   0
nerc region                0
climate region             0
anomaly level              0
climate category           0
cause category             0
outage duration (min)      0
demand loss mw           705
customers affected       443
population                 0
start time                 9
restoration time          58
total time                58
outage duration (hrs)      0
dtype: int64

In [29]:
# print(df.head().to_markdown(index=False))

### Baseline Model

We will start by splitting the data into training and testing sets. We will then create a basic `DecisionTreeClassifier()` to classify the cause of the outage based on `climate region` and `climate category`. The `climate region` refers to regions designated by National Centers for Environmental Information (Northeast, South, West, etc). The `climate category` column is based on the ONI El Niño/La Niña index that represents how warm/cold a season is, and is the index based off the total year. 

Extreme `climate category` is more likely to correlate with extreme weather and `climate region` can also be an indicator of weather habits that affect outages. Additionally, public officials may deliberately choose to shut off the power grid during certain weather patterns.

In [5]:
x = df.drop(['cause category'], axis = 1)
y = df['cause category']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [6]:
dt_base = DecisionTreeClassifier(max_depth = 3)

base_pl = Pipeline([('base', ColumnTransformer(
            [('climate category', OneHotEncoder(), ['climate category']), 
             ('climate region', OneHotEncoder(), ['climate region'])],
            remainder = 'drop')), 
        ('basic tree', dt_base)
    ])

base_pl.fit(x_train, y_train);

# we will calculate the score using the built-in .score() method. this is accuracy
print('train accuracy: ' + str(base_pl.score(x_train, y_train)))
print('test accuracy: ' + str(base_pl.score(x_test, y_test)))


train accuracy: 0.5921739130434782
test accuracy: 0.5234375


This doesn't seem completely awful, but it's not anywhere near good either. `base_pl.score()` calls upon `DecisionTreeClassifier()`'s `.score()` method, which returns accuracy. We will find the F-1 score next using weighted averages.

In [7]:
print('train F-1 score: ' + str(f1_score(y_train, base_pl.predict(x_train), average = 'weighted')))
print('test F-1 score: ' + str(f1_score(y_test, base_pl.predict(x_test), average = 'weighted')))

train F-1 score: 0.49595961205501965
test F-1 score: 0.4178350638155524


This seems a little worse. When only looking at the accuracy (how many true positives), this base model was able to predict the correct power outage around 55% of the time. The F-1 score shows it to be worse, however. While I wouldn't classify this model as good, I wouldn't classify it as awful either. Clearly the model can be improved. 

### Final Model

Now, we want to create a better model. In addition to the two features looked at previously, we will also look at `us state`, `anomaly level`, and `outage duration (min)`. We will perform a quantile encoding for `outage duration (min)` and map `us state` to a numerical value using a custom function.

Additionally, we will also search for the best hyperparameters to use in this model. We will tune the maximum depth of the tree, the minimum number of samples needed to split an internal node, and the criterion for splitting (the function by which the quality of the split is judged). 

- Too deep of a tree may result in overfitting; not deep enough does not allow for detailed enough classification
- The larger the minimum number of samples needed to split an internal node, the more likely the classifier will generalize
- Different criteria for splitting will lead to different decisions and classifications; the ones chosen are the accepted criteria as given in the documentation 

In [8]:
states = dict(zip(df.groupby('us state').mean().reset_index()['us state'].values, [i for i in range(1, 51)]))

def encode_state(df):
    df['us state'] = df['us state'].apply(lambda x: states[x])
    return df

param_tree = DecisionTreeClassifier()

param_pl = Pipeline([('base', ColumnTransformer(
            [('one_hot', OneHotEncoder(), ['climate category', 'climate region']),
             ('state', FunctionTransformer(encode_state), ['us state']),
             ('anomaly', FunctionTransformer(lambda x: x), ['anomaly level']),
             ('outage_duration', QuantileTransformer(n_quantiles = 900), ['outage duration (min)'])],
            remainder = 'drop')), 
        ('param_tree', param_tree)
    ])

# had to limit the number of quantiles as the default is larger than the number of samples used for training 
# per k-fold

hyperparameters = {
    'param_tree__max_depth': [2, 3, 4, 5, 7, 10, None], 
    'param_tree__min_samples_split': [2, 5, 7, 10],
    'param_tree__criterion': ['gini', 'entropy']
}

searcher = GridSearchCV(param_pl, hyperparameters, cv = 8)
searcher.fit(x_train, y_train);
searcher.best_params_

{'param_tree__criterion': 'entropy',
 'param_tree__max_depth': 5,
 'param_tree__min_samples_split': 2}

We will use the above parameters for our final model as determined by the `GridSearchCV`

In [9]:
final_tree = DecisionTreeClassifier(criterion = 'entropy', max_depth = 5, min_samples_split = 2)

final_pl = Pipeline([('base', ColumnTransformer(
            [('one_hot', OneHotEncoder(), ['climate category', 'climate region']),
             ('state', FunctionTransformer(encode_state), ['us state']),
             ('anomaly', FunctionTransformer(lambda x: x), ['anomaly level']),
             ('outage_duration', QuantileTransformer(n_quantiles = 900), ['outage duration (min)'])],
            remainder = 'drop')), 
        ('param_tree', final_tree)
    ])

final_pl.fit(x_train, y_train)

print('final train accuracy: ' + str(final_pl.score(x_train, y_train)))
print('final test accuracy: ' + str(final_pl.score(x_test, y_test)))

print('final train F-1 score: ' + str(f1_score(y_train, final_pl.predict(x_train), average = 'weighted')))
print('final test F-1 score: ' + str(f1_score(y_test, final_pl.predict(x_test), average = 'weighted')))

final train accuracy: 0.72
final test accuracy: 0.6302083333333334
final train F-1 score: 0.7025791136434512
final test F-1 score: 0.6015811365023541


We can compare this to the original base model.

In [10]:
print('base train accuracy: ' + str(base_pl.score(x_train, y_train)))
print('base test accuracy: ' + str(base_pl.score(x_test, y_test)))

print('base train F-1 score: ' + str(f1_score(y_train, base_pl.predict(x_train), average = 'weighted')))
print('base test F-1 score: ' + str(f1_score(y_test, base_pl.predict(x_test), average = 'weighted')))

base train accuracy: 0.5921739130434782
base test accuracy: 0.5234375
base train F-1 score: 0.49595961205501965
base test F-1 score: 0.4178350638155524


In comparing both F-1 and accuracy scores from the base model and from the final model, it is clear that the final model performs much better. 

### Fairness Analysis

Now we see if the model performs better or worse for certain groups. We will split our data by state into landlocked states and not landlocked states. A quick search (or if a map if you are exceptionally good at geography) tells us that the not landlocked states are: Alaska, Hawai'i, Washington, Oregon, California, Texas, Louisiana, Alabama, Florida, Georgia, South Carolina, North Carolina, Virginia, Maryland, Delaware, New Jersey, Mississippi, New York, Connecticut, Rhode Island, Massachusetts, New Hampshire, and Maine. Two notes on this: 
- Rhode Island has somehow avoided any power outages (or its power outages avoided being included in this set), so we will not include it
- Hawai'i is written as Hawaii in this dataset; we will use Hawaii

In [32]:
# split the data according to landlocked and not landlocked

not_locked = ['Alaska', 'Hawaii', 'Washington', 'Oregon', 'California', 'Texas', 'Louisiana', 'Alabama',
              'Florida', 'Georgia', 'South Carolina', 'North Carolina', 'Virginia', 'Maryland', 'Delaware',
              'New Jersey', 'Mississippi', 'New York', 'Connecticut', 'Rhode Island', 'Massachusetts',
              'New Hampshire', 'Maine']

df['landlocked'] = df['us state'].apply(lambda x: 0 if x in not_locked else 1)

df.head()

,us state,nerc region,climate region,anomaly level,climate category,cause category,outage duration (min),demand loss mw,customers affected,population,start time,restoration time,total time,outage duration (hrs),landlocked
0,Minnesota,MRO,East North Central,-0.3,normal,severe weather,3060.0,NaN,70000.0,5348119.0,2011-07-01 17:00:00,2011-07-03 20:00:00,2 days 03:00:00,51.00,1
1,Minnesota,MRO,East North Central,-0.1,normal,intentional attack,1.0,NaN,NaN,5457125.0,2014-05-11 18:38:00,2014-05-11 18:39:00,0 days 00:01:00,0.02,1
2,Minnesota,MRO,East North Central,-1.5,cold,severe weather,3000.0,NaN,70000.0,5310903.0,2010-10-26 20:00:00,2010-10-28 22:00:00,2 days 02:00:00,50.00,1
3,Minnesota,MRO,East North Central,-0.1,normal,severe weather,2550.0,NaN,68200.0,5380443.0,2012-06-19 04:30:00,2012-06-20 23:00:00,1 days 18:30:00,42.50,1
4,Minnesota,MRO,East North Central,1.2,warm,severe weather,1740.0,250.0,250000.0,5489594.0,2015-07-18 02:00:00,2015-07-19 07:00:00,1 days 05:00:00,29.00,1


In [34]:
# print(df.head().to_markdown(index=False))

We now create our permutation test to see if the model performs better or worse for landlocked states. Our null and alternative hypotheses will be as follows:

- $H_0$ (null): The final model is fair. Its F-1 score for both landlocked states and not landlocked states are roughly the same and any difference is due to chance.
- $H_a$ (alternative): The final model is unfair. The F-1 score for landlocked states and not landlocked states are not equal.

We will repeatedly shuffle the `landlocked` column, split the data into landlocked and not landlocked, and conduct a permutation test based on the absolute difference between the F-1 scores for each group. We will use a p-value of 0.05.

In [21]:
locked = df[df['landlocked'] == 1]
not_locked = df[df['landlocked'] == 0]

locked_f1 = f1_score(locked['cause category'], final_pl.predict(locked.drop('cause category', axis = 1)), 
                     average = 'weighted')
not_locked_f1 = f1_score(not_locked['cause category'], 
                     final_pl.predict(not_locked.drop('cause category', axis = 1)), 
                     average = 'weighted')

obs_diff = np.abs(locked_f1 - not_locked_f1)

n = 1000
diffs = []

for i in range(n):
    
    with_shuffled = df.assign(locked_shuff = np.random.permutation(df['landlocked']))
    
    locked = with_shuffled[with_shuffled['locked_shuff'] == 1]
    not_locked = with_shuffled[with_shuffled['locked_shuff'] == 0]

    locked_f1 = f1_score(locked['cause category'], final_pl.predict(locked.drop('cause category', axis = 1)), 
                     average = 'weighted')
    not_locked_f1 = f1_score(not_locked['cause category'], 
                     final_pl.predict(not_locked.drop('cause category', axis = 1)), 
                     average = 'weighted')

    diff = np.abs(locked_f1 - not_locked_f1)
    diffs.append(diff)

In [22]:
fig = px.histogram(pd.DataFrame(diffs), x = 0, nbins = 50, histnorm = 'probability', 
                   title = 'Empirical Distribution of the Absolute Difference')

fig.add_vline(x = obs_diff, line_color = 'lightseagreen')
fig.add_annotation(text = f'<span style="color:lightseagreen">Observed Absolute Difference</span>',
                   x = 1.9 * obs_diff, showarrow = False, y = 0.055)

fig.update_layout(xaxis_range = [0, np.max(diffs) + 0.01])
fig.update_layout(font_family = "sans-serif")
fig.update_traces(marker_color = 'pink')

fig.write_html('abs diff.html', include_plotlyjs = 'cdn')
fig.show()

In [23]:
p_val = np.mean(np.array(diffs) > obs_diff)
p_val

0.508

Therefore, we fail to reject our null hypothesis that the final model is fair. It is most likely, based on the data and tests, that our final model does not find a distinction when grouping based off the landlocked status of the state in which the power outage occurred. 